# Signal Windowing

In [2]:
import pandas as pd
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons, TextBox
import time
from IPython.display import clear_output
import functionsMasterProjectMeinhart as fmpm

# Training Data

In [3]:

# load all date, except data from one subject (test data)
test_data_subject = 9

db_name='DataBase_Pysio.db' # database name

exercise_abbrs = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2'] # exercise abbreviations


# Connect to an existing database
conn = sqlite3.connect(db_name)
cur = conn.cursor()

# dictionary with the exercise abbreviation as key
data_points = {}

for key in exercise_abbrs:
    # sql command to extract data
    query_sql = """
        SELECT r.start_time, r.stop_time, e.csv_file
        FROM subjects s
        INNER JOIN exercises e
        ON s.id = e.subject_id
        INNER JOIN paradigms p
        ON p.id = e.paradigm_id
        INNER JOIN repetitions r
        ON e.id = r.exercise_id
        WHERE p.abbreviation = '{}'
        AND NOT s.id = {}
        """.format(key, test_data_subject)

    # get data from data base and close connection
    data_points[key] = pd.read_sql_query(query_sql, conn)
    
conn.close()

In [4]:
print('Head of one loaded data frame as an example:')
data_points['RF'].head()

Head of one loaded data frame as an example:


,start_time,stop_time,csv_file
0,1.1482924107142871,3.699122023809525,subject01_RF_05.csv
1,3.699122023809525,6.49581473214286,subject01_RF_05.csv
2,6.49581473214286,9.384706101190478,subject01_RF_05.csv
3,9.384706101190478,12.073833705357146,subject01_RF_05.csv
4,12.073833705357146,14.809060639880954,subject01_RF_05.csv


In [5]:
print('Length of the individual data frames:')
count = 0
for key in exercise_abbrs:
    print(key + ':\t' + str(data_points[key].shape[0]))
    count += data_points[key].shape[0]
print('total:\t' + str(count))

Length of the individual data frames:
RF:	239
RO:	240
RS:	240
LR:	240
BC:	243
TC:	242
MP:	242
SA:	243
P1:	240
P2:	240
total:	2409


In [6]:
def split_range_into_sections(signal_data, num_sec=10, signals=['Acc','Gyr'], start_index=0, stop_index=None):
    '''
    This function splits a selected range of the input signals into a defined number 
    of equally distributed sections. For each signal and section the mean is calculated,
    and afterwards returned by means of a dictionary.
    
    Parameters
    ----------
    signal_data : dict
        Dictionary with the signals in the 'signals' argument as keys.
        The signal arrays must have same length.
    
    num_sec : int
        Number of sections to split the signals.
        
    signals : list
        Keys to select signals in the signal_data dictionary.
        If no keys are provided, all keys of the signal_data
        dictionary are taken.
        
    start_index : int
        Start index of selected range (default=0).
    
    stop_index : int
        Stop index of selected range.
        If not given --> take length of signal data.
    
    Returns
    -------
    Dictionary with section means for each signal --> keys are same as the selected
    in the list "signals".
    '''
    
    # if no signals are given as keys, select all keys of the input dictionary
    if not signals:
        signals = [*signal_data]
    
    # number of input data points of each signal (signals have to be of the same length --> take index 0)
    len_signals = np.shape(signal_data[signals[0]])[0]
    
    # check if stop index is given
    if stop_index is None:
        stop_index = len_signals
    
    # get indices of the sections (+1 due to start and end index of each section)
    sec_ind = np.linspace(start_index, stop_index, num_sec+1).round().astype(int)
    
    # dicitonary to save sections means for each signal
    section_means = {}

    for signal in signals:
        # generate row with zeros in order to use np.vstack afterwards
        section_means[signal] = np.zeros([1, np.shape(signal_data[signal])[1]])

        # add the mean of each section
        for ii in range(len(sec_ind)-1):
            section_means[signal] = np.vstack([section_means[signal], 
                                               np.mean(signal_data[signal][sec_ind[ii]:sec_ind[ii+1]], axis=0)])

        # delete the first row with the zeros
        section_means[signal] = np.delete(section_means[signal], 0, axis=0)
        
    return section_means

## Generate and save features

In [7]:
number_sections = 10 # number of sections to split the signals

csv_dir='E:\Physio_Data_Split_Exercise_done' # directory of csv files
#  csv-file to save the features
feature_csv_file = '1_ML_Approach_Features/features_without_sub9_sections_'+'{:02}'.format(number_sections)+'.csv' 

sampling_rate = 256 # [Hz]
signals = ['Acc','Gyr'] # signals which shall be considered for the mean calculation

***Execute cell below only if csv-file does not already exist***

In [ ]:
# putting the header of the feature-file together
header_string = 'label;' # first column contains the labels

for sig in signals:
    for ax in ['_x','_y','_z']:
        for ii in range(number_sections):
            header_string +=  sig + ax + '_{:02}'.format(ii+1) + ';'

# remove last separator (;)
idx_last_sep = header_string.rfind(";")
header_string =  header_string[:idx_last_sep]

# write header to file
with open(feature_csv_file, 'w') as feature_file:
    feature_file.writelines(header_string + '\n')


# go through all exercises
for ex in exercise_abbrs:
    
    # go through all repetitions (data points) of the current exercise
    for ii in range(len(data_points[ex])):
        
        # join file path
        file_path = os.path.join(csv_dir, data_points[ex]['csv_file'][ii])
        
        # load the signal data of the corresponding time range of the current repetition
        selected_data = fmpm.get_sensor_data(in_file = file_path, 
                                             signals = signals, 
                                             sampling_rate = sampling_rate, 
                                             start_time = float(data_points[ex]['start_time'][ii]), 
                                             stop_time = float(data_points[ex]['stop_time'][ii]))
         
        # calculate the corresponding section means of the current repetition
        section_means = split_range_into_sections(signal_data = selected_data,
                                                  num_sec = number_sections,
                                                  signals = signals)
        
        # string to write data of the current data point to the csv-file
        data_point_string = ex + ';' # first column contains the label
        
        # copy section mean values to string
        for sig in signals:
            for jj in [0,1,2]: # x, y, z comp. of the corresponding signal
                for ll in range(number_sections):
                    
                    # append to string for writing to csv file (5 decimals)
                    data_point_string += "{:.5f};".format(section_means[sig][ll,jj])
                    
        # remove last separator (;)
        idx_last_sep = data_point_string.rfind(";")
        data_point_string =  data_point_string[:idx_last_sep]
        
        # append values of current data point to file
        with open(feature_csv_file, 'a') as feature_file:
            feature_file.writelines(data_point_string + '\n')
                    

## Load the generated features

In [8]:
feature_data = pd.read_csv(feature_csv_file, skiprows=0, sep=';')
feature_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 61 columns):
label       2409 non-null object
Acc_x_01    2409 non-null float64
Acc_x_02    2409 non-null float64
Acc_x_03    2409 non-null float64
Acc_x_04    2409 non-null float64
Acc_x_05    2409 non-null float64
Acc_x_06    2409 non-null float64
Acc_x_07    2409 non-null float64
Acc_x_08    2409 non-null float64
Acc_x_09    2409 non-null float64
Acc_x_10    2409 non-null float64
Acc_y_01    2409 non-null float64
Acc_y_02    2409 non-null float64
Acc_y_03    2409 non-null float64
Acc_y_04    2409 non-null float64
Acc_y_05    2409 non-null float64
Acc_y_06    2409 non-null float64
Acc_y_07    2409 non-null float64
Acc_y_08    2409 non-null float64
Acc_y_09    2409 non-null float64
Acc_y_10    2409 non-null float64
Acc_z_01    2409 non-null float64
Acc_z_02    2409 non-null float64
Acc_z_03    2409 non-null float64
Acc_z_04    2409 non-null float64
Acc_z_05    2409 non-null float64
Acc_z_06  

In [9]:
feature_data.head()

,label,Acc_x_01,Acc_x_02,Acc_x_03,Acc_x_04,Acc_x_05,Acc_x_06,Acc_x_07,Acc_x_08,Acc_x_09,...,Gyr_z_01,Gyr_z_02,Gyr_z_03,Gyr_z_04,Gyr_z_05,Gyr_z_06,Gyr_z_07,Gyr_z_08,Gyr_z_09,Gyr_z_10
0,RF,-0.94158,-0.84232,-0.06218,0.52175,0.58297,0.22722,-0.43416,-0.99045,-0.97328,...,103.51442,174.14299,124.47019,38.04615,-11.82500,-100.36080,-155.38269,-134.52115,-63.53977,-4.68558
1,RF,-0.92893,-1.00249,-0.48374,0.21750,0.57440,0.36039,-0.31218,-0.95081,-0.93390,...,38.50087,139.13732,152.86458,90.02641,11.11632,-58.15191,-155.85739,-144.87066,-66.45246,3.56337
2,RF,-0.89641,-0.94826,-0.36936,0.29406,0.53504,0.32506,-0.23003,-0.86684,-0.95014,...,46.91807,133.67483,143.52111,59.91554,8.84122,-61.00942,-131.24409,-142.97044,-68.36571,3.00929
3,RF,-0.92039,-0.97978,-0.60092,0.12516,0.49953,0.38173,-0.17635,-0.73176,-0.95656,...,33.52264,126.68025,152.88496,100.61685,13.80423,-64.13678,-122.12228,-140.03351,-91.74547,-2.57428
4,RF,-0.93697,-0.86661,-0.26934,0.26927,0.54755,0.39191,-0.08301,-0.61854,-0.90253,...,63.20268,139.25179,130.30089,63.78125,6.25268,-47.06964,-110.11339,-116.17143,-95.74554,-13.73214


In [10]:
# get feature matrix
X_original = feature_data.values[:,1:]
np.shape(X_original)

(2409, 60)

In [11]:
# dictionary for labels
label_ex = {'RF':0,
            'RO':1,
            'RS':2,
            'LR':3,
            'BC':4,
            'TC':5,
            'MP':6,
            'SA':7,
            'P1':8,
            'P2':9}

# get label array with labels (0 ... 9)
y = [label_ex[feature_data.values[ii,0]] for ii in range(len(feature_data.values[:,0]))]
np.shape(y)

(2409,)

In [12]:
# get train data of separate exercises

exercise_abbrs = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2'] # exercise abbreviations

X_ex = {} # dicitionary for separate feature matrices

for ex in exercise_abbrs:
    
    ind = [ii for ii, ex_num in enumerate(y) if ex_num==label_ex[ex]] # indices for the current exercise

    X_ex[ex] = feature_data.values[ind,1:]
    

In [17]:
np.shape(X_ex['RO'])

(240, 60)

## ML models to distinguish between exercise and non-exercise

In [19]:
from sklearn.neighbors import LocalOutlierFactor

In [63]:
# generate one LocalOutlierFactor model for each exercise
distExNonex = {}
for ex in exercise_abbrs:
    distExNonex[ex] = LocalOutlierFactor(n_neighbors=20, novelty=True, contamination='auto')
    distExNonex[ex].fit(X_ex[ex])

# method: score_samples
# returns: opposite_lof_scores : array, shape (n_samples,)
# --> The opposite of the Local Outlier Factor of each input samples. The lower, the more abnormal.


In [43]:
plt.plot(X_ex['RF']);

*** Random Forest Classifier***

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
# create random forest classifier
rnd_clf_orig = RandomForestClassifier(n_estimators=500, max_leaf_nodes=40, n_jobs=-1, random_state=42)

# train the model
rnd_clf_orig.fit(X_original, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=40,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

## Loading test data

In [46]:
# select file (csv) of test subject
file_dir  = r'E:\Physio_Data\Subject_09'
file_name = 'subject09.csv'

# selecet time range [min:sec]
#start_min_sec = '01:36.0' # Raises Oblique
#stop_min_sec  = '02:00.3'

# selecet time range [min:sec]
start_min_sec = '00:00.0'
stop_min_sec  = '35:00.0'

sampling_rate = 256 # [Hz]

# split time string and convert to float
start_min = float(start_min_sec.split(':')[0])
start_sec = float(start_min_sec.split(':')[1])
stop_min = float(stop_min_sec.split(':')[0])
stop_sec = float(stop_min_sec.split(':')[1])

# start and stop time in seconds
start_time = start_min*60 + start_sec # [s]
stop_time = stop_min*60 + stop_sec # [s]

# join data path
data_path = os.path.join(file_dir, file_name)

# get data from selected file
sensor_data = fmpm.get_sensor_data(in_file=data_path,
                                   sampling_rate=sampling_rate,
                                   start_time=start_time,
                                   stop_time=stop_time)

# filter properties
cutoff = 10 # [Hz]
order = 6 # butterworth order

# filter data with butterworth filter and save to new dictionary
signal_keys = ['Acc', 'Gyr']
sensor_data_filt = {}
for signal in signal_keys:
    sensor_data_filt[signal] = fmpm.butter_lowpass_filter(sensor_data[signal], 
                                                          cutoff=cutoff, 
                                                          fs=sampling_rate, 
                                                          order=order)

In [47]:
def get_window_indices(signal_len, window_length=5, start_time=None, start_index=0, sampling_rate=256, auto_end=True):
    '''
    This function returns the indices of a certain time range,
    selected via start time (or start index) and window length.
    
    Parameters
    ----------
    signal_len : int
        signal length (indices)
        
    window_length : float or int
        window length in seconds to select data
        
    start_time : float or int
        start time in seconds where data selection starts
        
    start_index : int
        only used, if start_time is None;
        start index where data selection starts
        
    sampling_rate : float or int
        sampling rate of signal data
        
    auto_end : boolean
        if True --> set stop index to signal length if out of range
    
    Returns
    -------
    list
        start and stop indices,
        or empty list if stop index is out of range and auto_end = False
    '''
    
    # if start time is given --> convert to start index
    if start_time is not None:
        start_index = int(start_time * sampling_rate)
    
    # calculate stop index
    stop_index = start_index + int(window_length * sampling_rate)
    
    if stop_index > signal_len:
        if auto_end is True:
            stop_index = signal_len
        else:
            return []
    
    return [start_index, stop_index]

## Windowing the test data

In [13]:
from IPython.display import HTML
HTML('<img src="windowing_procedure.gif" width=600 >')

In [48]:
# window start increment
win_start_inc = 0.2 # [s]

# window stretch increment
win_stretch_inc = 0.2 # [s]

# minimum window length
win_min_len = 1 # [s]

# maximim window length
win_max_len = 5 # [s]

# sampling rate of the signals
sampling_rate = 256 # [Hz]

# signal names
sig_names = ['Acc','Gyr']

# number of sections to split the signal
number_sections = 10

# signal length (all sensor data must have same length --> Acc, Gyr, ...)
signal_len = len(sensor_data[sig_names[0]])

# 
## selecet time range [min:sec]
#win_start_min_sec = '30:00.0'
#win_last_start_min_sec  = '34:45.0'
## split time string and convert to float
#win_start_min = float(win_start_min_sec.split(':')[0])
#win_start_sec = float(win_start_min_sec.split(':')[1])
#win_last_start_min = float(win_last_start_min_sec.split(':')[0])
#win_last_start_sec = float(win_last_start_min_sec.split(':')[1])
## start and stop time in seconds
#win_start = win_start_min*60 + win_start_sec # [s]
#win_last_start = win_last_start_min*60 + win_last_start_sec # [s]

# ---> selsect all data
# window start time
win_start = 0 # [s]

# last window start time --> time where the minimum window length just fits into the sensor data
win_last_start = signal_len/sampling_rate - win_min_len




# number of different window sizes
num_win_sizes = len(np.arange(win_min_len, win_max_len+win_stretch_inc, win_stretch_inc))

# number of different window start points
num_start_points = len(np.arange(win_start, win_last_start, win_start_inc))

# matrix to save the distinction values (exercise/non-exercise)
dist_matrix = np.zeros([num_start_points, num_win_sizes])

# matrices to save predicted values for all classes
# exercise_abbrs = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2']
dist_matrix = {}
for ex in exercise_abbrs:
    dist_matrix[ex] = np.zeros([num_start_points, num_win_sizes])

    
feature_map = np.zeros([num_start_points * num_win_sizes, number_sections*6])

In [64]:
# function to print progress

def print_progress(current_num, max_num, prev_prog):
    
    new_prog = int(current_num/max_num*100)
    
    if new_prog > prev_prog:
        clear_output(wait=True)
        print('Progress: {:3d}%'.format(new_prog))
        
    return new_prog

In [65]:
count = 0
max_count = len(feature_map)
prev_progress = 0 # previous progress

# going through all window start points
for ii, win_pos in enumerate(np.arange(win_start, win_last_start, win_start_inc)):

    # going through all window lengths  (+win_stretch_inc to include end point)
    for jj, win_len in enumerate(np.arange(win_min_len, win_max_len+win_stretch_inc, win_stretch_inc)):

        [idx_start, idx_stop] = get_window_indices(signal_len, 
                                                   window_length = win_len, 
                                                   start_time = win_pos,
                                                   sampling_rate = sampling_rate, 
                                                   auto_end = True)

        section_means = split_range_into_sections(sensor_data, 
                                                  num_sec = number_sections, 
                                                  signals = sig_names, 
                                                  start_index = idx_start, 
                                                  stop_index = idx_stop)
        
        
        feature_map[count,:] = np.concatenate((section_means[sig_names[0]].transpose(), 
                                               section_means[sig_names[1]].transpose())).flatten().reshape(1, -1)
        
        count += 1
        
    prev_progress = print_progress(count, max_count, prev_progress)
        
        
        #pred_probs = rnd_clf_orig.predict_proba(features)[0]
        
        #for kk, ex in enumerate(exercise_abbrs):
        #    dist_matrix[ex][ii,jj] = pred_probs[kk]
        
        #dist_matrix[ii, jj] = distExNonex.score_samples(features)[0]


Progress: 100%


In [66]:
count = 0
max_count = 10 # 10 models
prev_progress = 0 # previous progress

scores_LOF = {}
for ex in exercise_abbrs:
    scores_LOF[ex] = distExNonex[ex].score_samples(feature_map)
    count += 1
    prev_progress = print_progress(count, max_count, prev_progress)

Progress: 100%


In [19]:
# Random Forest Classifier
#pred_probs = rnd_clf_orig.predict_proba(feature_map)
#np.shape(pred_probs)

(220395, 10)

In [68]:
# creating the images

count = 0
max_count = len(feature_map)
prev_progress = 0 # previous progress

# going through all window start points
for ii, win_pos in enumerate(np.arange(win_start, win_last_start, win_start_inc)):
    # going through all window lengths  (+win_stretch_inc to include end point)
    for jj, win_len in enumerate(np.arange(win_min_len, win_max_len+win_stretch_inc, win_stretch_inc)):
        
        for kk, ex in enumerate(exercise_abbrs):
            # dist_matrix[ex][ii,jj] = pred_probs[count][kk] # for RFC probabilities
            dist_matrix[ex][ii,jj] = scores_LOF[ex][count]
            
        count += 1
        prev_progress = print_progress(count, max_count, prev_progress)
        

Progress: 100%


In [69]:
def convert_time_format_to_index(min_sec, sampling_rate, time_offset=0, max_index=None):
    '''
    Function converts a string with the time format 'min:sec' (e.g. 5:17.2)
    to a corresponding index, considering the sampling rate.
    If index would be negative, 0 is returned.
    
    Inputs
    ------
    min_sec : string
        Time data, defined format: 'min:sec'
    
    sampling_rate : float or int
        Sampling rate for the index calculation. [Hz]
        
    time_offset : float of int
        Time offset, considered at the index calculation. [s]
        
    max_index : int
        Maximum valid index.
        If provided and calculated index is out of range,
        max_index is returned instead.
     
    Returns
    -------
    int
        Corresponding index to parameter 'min_max'.
    '''
    
    # split time string and convert to float
    minutes = float(min_sec.split(':')[0])
    seconds = float(min_sec.split(':')[1])
    
    # start and stop time in seconds
    time_s = minutes*60 + seconds + time_offset
    
    # get corresponding index
    index = round(time_s * sampling_rate)
    
    # ensure that index is not below 0
    if index < 0:
        index = 0
    
    # ensure that index is in valid range if max index is given
    if max_index is not None and index > max_index:
        index = max_index
            
    return index

## Plot the results

In [70]:
plt.plot(scores_LOF['MP']);

In [71]:
%matplotlib auto


yticks = np.arange(0, win_max_len-win_min_len+win_stretch_inc, 1) / win_stretch_inc
ylabels = ['{}'.format(yticks[ii] * win_stretch_inc + win_min_len) for ii in range(len(yticks))]


fig, axis = plt.subplots(11,1,figsize=(18,8), sharex=True)
fig.suptitle('Scores\nLocal Outlier Factor', fontsize=20)

# image color settings for RFC
#vmin=0
#vmax=1

# image color settings for LOF
vmin=-3
vmax=-1

for ax, ex in zip(axis, exercise_abbrs):
    s = ax.imshow(dist_matrix[ex].transpose(), interpolation='nearest', 
                  aspect='auto', cmap=plt.cm.seismic, vmin=vmin, vmax=vmax)
    ax.invert_yaxis()
    ax.set_yticks(yticks)
    ax.set_yticklabels(ylabels)
    ax.set_ylabel(ex, fontsize=13)
    ax.xaxis.set_ticklabels([])

axis[-1].plot(range(num_start_points), np.zeros(num_start_points), 'k', alpha=0.0)
formatter = matplotlib.ticker.FuncFormatter(lambda i, x: time.strftime('%M:%S', time.gmtime(i*win_start_inc+win_start)))
axis[-1].xaxis.set_major_formatter(formatter)
axis[-1].set_yticks([])
axis[-1].set_ylim([0,1])
axis[-1].set_xlabel(r'time $[min:sec]$', fontsize=13)

fig.subplots_adjust(bottom=0.2, right=0.9) # make space for buttons and color bar
cbar_ax = fig.add_axes([0.92, 0.1, 0.01, 0.8])
fig.colorbar(s, cax=cbar_ax)


# add slider for selections on the x axis
Slider_shiftX_ax = plt.axes([0.1, 0.07, 0.75, 0.025])
Slider_zoomX_ax = plt.axes([0.1, 0.035, 0.75, 0.025])

axcolor = 'cornflowerblue'
Slider_shiftX = Slider(Slider_shiftX_ax, 'x shift [%]', 0.0, 100.0, valinit=0, facecolor=axcolor)
Slider_zoomX = Slider(Slider_zoomX_ax, 'x scale [%]', 0.1, 100.0, valinit=100, facecolor=axcolor)
Slider_zoomX_ax.xaxis.set_visible(True)
Slider_zoomX_ax.set_xticks(np.arange(0,105,5)) 


def updateX(val):
    start_index = int(Slider_shiftX.val / 100 * num_start_points)
    stop_index = start_index + Slider_zoomX.val / 100 * num_start_points
    axis[-1].set_xlim((start_index, stop_index))
    plt.draw()

Slider_shiftX.on_changed(updateX)
Slider_zoomX.on_changed(updateX)


# add textboxes to select time range
initial_start_text = "00:00.0"
initial_stop_text = "00:00.0"


def updateXval(text):
    fig.suptitle(text_box_stop.text)
    try:
        start_index = convert_time_format_to_index(text_box_start.text, sampling_rate=1/win_start_inc, 
                                                   time_offset=0, max_index=None)
        stop_index = convert_time_format_to_index(text_box_stop.text, sampling_rate=1/win_start_inc, 
                                                  time_offset=0, max_index=None)
    except:
        return 0
    
    else:
        if start_index < stop_index:
            axis[-1].set_xlim((start_index, stop_index))
            plt.draw()
        
axbox_start = plt.axes([0.1, 0.12, 0.05, 0.03])
text_box_start = TextBox(axbox_start, 'Start time: ', initial=initial_start_text)
text_box_start.on_submit(updateXval)

axbox_stop = plt.axes([0.2, 0.12, 0.05, 0.03])
text_box_stop = TextBox(axbox_stop, 'Stop time: ', initial=initial_stop_text)
text_box_stop.on_submit(updateXval)


axis[-1].set_xlim(0, num_start_points)

plt.show()


Using matplotlib backend: TkAgg
